<h1 align="center"> Computation for Physicists </h1>
<h2 align="center"> <em> Extend Python with C++</em> </h2>
<h2 align="center" > <a href="mailto:duan@unm.edu">Dr. Duan</a> (UNM) </h2>

# Python vs C++

- Python is an "interpreted" language; C++ programs need to be precompiled.
    - Both are portable, but Python is cross-platform executable.
    - C++ codes run much faster than Python codes.

- Python variables are labels; C++ variables "are" memory locations.

```c++
// C++
int a = 10; // declare the type of variable   
a = 9; // only the same data can be assigned
```

```python
# Python
a = 10 # no need to declare variable
a = 'b' # can be "assigned" with value of a different type
```

- Python uses dynamic typing; C++ ueses static typing.

In [1]:
# Python
import numpy as np # import NumPy package
def times2(x): # type of x is unknown
    return x * 2

times2(2), times2(2.0), times2('2'), times2([1, 1]), times2(np.array([1, 1]))


(4, 4.0, '22', [1, 1, 1, 1], array([2, 2]))

```c++
// C++, polymorphism
int times2(int x) { return x * 2; }
double times2(double x) { return x * 2;}
...
```

```c++
// C++, template
template <class T> times2(T x) { return x*2; }

int a = times2<int>(2);
```

- Dynamic typing: If it quacks, it's a duck!

In [2]:
class vec: # 3D vector class
    def __init__(self, x, y, z): # construct the object
        self.x = x; self.y = y; self.z = z

    def __repr__(self): # format the output
        return f'({self.x}, {self.y}, {self.z})'

    def __mul__(self, c): # overload operator *
        return vec(self.x*c, self.y*c, self.z*c)

a = vec(1, '1', [1.0])
times2(a)

(2, 11, [1.0, 1.0])

# Combine Python and C++

- Python codes are easy to write and debug; C++ codes run fast.

- A possible way to combine the advantages of both: Write the interface and the prototype in Python and the production core in C++ ==> [pybind11](https://pybind11.readthedocs.io)

## Homework 4 from last semester:

- The standard deviation of a dataset $\{x_1, x_2, \cdots, x_n\}$ is defined as
$$\sigma = \sqrt{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2},$$
where $\bar{x}$ is the average value of $x_i$. It is straightforward to show that an equivalent definition of the standard deviation is
$$\sigma = \sqrt{\left(\frac{1}{n}\sum_{i=1}^n x_i^2\right)- \bar{x}^2}.$$
Which of the two algorithms is potentially faster, and which is more accurate?

In [3]:
from mystat import mean_std, mean_std2, mean_std_np, mean_std2_np
import numpy as np
from numpy.random import random

n = 10**3 # number of values
data = 10+random(n) # dataset
(np.mean(data), np.std(data)), mean_std(data), mean_std2(data), \
mean_std_np(data), mean_std2_np(data)

((10.488727139931761, 0.2908702700944497),
 (10.488727139931768, 0.29087027009444993),
 (10.488727139931768, 0.2908702700942741),
 (10.488727139931761, 0.2908702700944497),
 (10.488727139931761, 0.2908702700944939))

## Benchmarks

In [4]:
%timeit mean_std(data)

888 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%timeit mean_std2(data)

744 µs ± 3.62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit mean_std_np(data)

37.7 µs ± 412 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
%timeit mean_std2_np(data)

30.7 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
%timeit np.mean(data)

18.5 µs ± 67.9 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
%timeit np.std(data)

60 µs ± 76.6 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## C++ Extension

In [10]:
import mystat_cpp

help(mystat_cpp.mean_std)

Help on built-in function mean_std in module mystat_cpp:

mean_std(...) method of builtins.PyCapsule instance
    mean_std(arg0: numpy.ndarray[numpy.float64]) -> Tuple[float, float]



In [11]:
mystat_cpp.mean_std(data), mystat_cpp.mean_std2(data)

((10.488727139931768, 0.29087027009444993),
 (10.488727139931768, 0.2908702700942741))

In [12]:
%timeit mystat_cpp.mean_std(data)

3.59 µs ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
%timeit mystat_cpp.mean_std2(data)

2.41 µs ± 6.96 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Parallelize with SIMD

- Single instruction, multiple data. E.g. $C[i] = A[i] + B[i]$, where $A$, $B$, and $C$ are 1D arrays.   

![](https://software.intel.com/content/dam/develop/external/us/en/images/37208-183287.gif)

Figure Credit: https://software.intel.com/content/www/us/en/develop/articles/introduction-to-intel-advanced-vector-extensions.html


In [14]:
import mystat_simd

mystat_simd.mean_std(data), mystat_simd.mean_std2(data)

((10.488727139931765, 0.29087027009444977),
 (10.488727139931765, 0.2908702700944695))

In [15]:
%timeit mystat_simd.mean_std(data)

2.37 µs ± 6.57 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [16]:
%timeit mystat_simd.mean_std2(data)

1.8 µs ± 0.981 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Parallelize with OpenMP

- Utilize multiple CPU cores with threads.

![](https://hpc.llnl.gov/tuts/openMP/images/fork_join2.gif)
Figure Credit: https://hpc.llnl.gov/tuts/openMP/

In [17]:
import mystat_omp

mystat_omp.mean_std(data), mystat_omp.mean_std2(data)

((10.488727139931761, 0.2908702700944497),
 (10.488727139931761, 0.2908702700945183))

In [18]:
%timeit mystat_omp.mean_std(data)

11.1 µs ± 11.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [19]:
%timeit mystat_omp.mean_std2(data)

8.74 µs ± 14.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Another Test

In [20]:
n = 10**7 # data size
data = 10+random(n) # dataset

In [21]:
%timeit mean_std_np(data)

46.9 ms ± 22.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%timeit mystat_cpp.mean_std(data)

25.5 ms ± 58.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit mystat_simd.mean_std(data)

15 ms ± 5.26 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit mystat_omp.mean_std(data)

4.4 ms ± 5.97 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
